In [1]:
## DO NOT RUN THIS.  It is used to build an updated BkAvgAst... table which is already in the processing zip file.
## first step in creating new BkAvgAst... file
## 2017/04/07:  updated for Mar 2017 failure data (Note: only failures before 2017 used since Call data as of Dec-2016)
## 2018/03/08: updated for Dec 2017 failure events (since Call Data only up to Dec17)
## 2018/08/27: updated failure rate details (number fails/number comm bks)
import numpy as np
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import os
os.chdir('C:/BankRisk')

df2 = pd.read_csv("df2_2001_201806.csv")
df2['AsOf'] = pd.to_datetime(df2['AsOf'])

In [2]:
fcol = df2.columns[5:]  ## get listing of numeric fields, then...
df2[fcol] = df2[fcol].astype(float)  ### CRITICAL to convert numeric data to FLOAT!!! (since some are ints)

In [3]:
df3 = df2.sort_index(by=['IDRSSD','AsOf'], ascending=True).reset_index(drop=True)
df3.head()

Unnamed: 0  IDRSSD       AsOf                    Name State     Cert  \
0           0      37 2001-03-31  BANK OF HANCOCK COUNTY    GA  10057.0   
1        8857      37 2001-06-30  BANK OF HANCOCK COUNTY    GA  10057.0   
2       17647      37 2001-09-30  BANK OF HANCOCK COUNTY    GA  10057.0   
3       26405      37 2001-12-31  BANK OF HANCOCK COUNTY    GA  10057.0   
4       35094      37 2002-03-31  BANK OF HANCOCK COUNTY    GA  10057.0   

   eTotAst   eTotLn   eALLL  eOREO   ...     ncoFIfor  ncoAg  ncoCIdom  \
0  65486.0  40507.0   979.0  200.0   ...          0.0    0.0       0.0   
1  65427.0  42153.0  1049.0    0.0   ...          0.0    0.0       0.0   
2  65575.0  44359.0  1070.0   40.0   ...          0.0    0.0       0.0   
3  66624.0  41514.0   987.0   20.0   ...          0.0    0.0       5.0   
4  68766.0  41539.0  1048.0   20.0   ...          0.0    0.0       0.0   

   ncoCIfor  ncoCard  ncoAuto  ncoOthCsmr  ncoForGovt  ncoOthLn  ncoLeas  
0       0.0      0.0      0.0        -1.0         0.0       0.0      0.0  
1       0.0      0.0      0.0       -23.0         0.0       0.0      0.0  
2       0.0      0.0      0.0         2.0         0.0       0.0      0.0  
3       0.0      0.0      0.0       104.0         0.0       0.0      0.0  
4       0.0      0.0      0.0       -13.0         0.0       0.0      0.0  

[5 rows x 196 columns]

In [4]:
len(df3) # 481187 (201612), 498723 (201709), 504444 (201712), 515693 (201806)

515693

In [5]:
'eDDep' in df3.columns

True

In [6]:
# get mean eTotAst by IDRSSD:
df4 = df3[['IDRSSD','AsOf','Name','State','Cert','eTotAst','aTotAst','eTotLn','eDDep','eFDep']]
df4['eTotDep'] = df4['eDDep'] + df4['eFDep']
df5 = df4[df4['eTotAst']>0].groupby('IDRSSD')

In [7]:
df6x = pd.DataFrame(df5['AsOf'].max()).reset_index()
df6n = pd.DataFrame(df5['AsOf'].min()).reset_index()

In [8]:
df6n.tail()

IDRSSD       AsOf
10767  5143788 2017-09-30
10768  5192496 2018-03-31
10769  5193989 2018-03-31
10770  5205819 2018-06-30
10771  5227101 2018-06-30

In [9]:
df4.tail()

IDRSSD       AsOf              Name State     Cert  eTotAst  aTotAst  \
515688  5192496 2018-06-30     ENDEAVOR BANK    CA  59099.0  50253.0  40512.0   
515689  5193989 2018-03-31     INFINITY BANK    CA  59098.0  34108.0  33477.0   
515690  5193989 2018-06-30     INFINITY BANK    CA  59098.0  43265.0  36734.0   
515691  5205819 2018-06-30       STUDIO BANK    TN  59112.0  45553.0  33282.0   
515692  5227101 2018-06-30  COMMERCEONE BANK    AL  59114.0  56804.0  51817.0   

         eTotLn    eDDep  eFDep  eTotDep  
515688  13263.0  26811.0    0.0  26811.0  
515689    285.0   4152.0    0.0   4152.0  
515690   4185.0  14155.0    0.0  14155.0  
515691      0.0    370.0    0.0    370.0  
515692   2170.0   8603.0    0.0   8603.0

In [10]:
df7 = df6x.merge(df4[['IDRSSD','AsOf','Name','State','Cert']],how='inner', left_on=['IDRSSD','AsOf'], right_on=['IDRSSD','AsOf'])
df7.columns = ['IDRSSD', 'xAsOf', 'xName', 'xState','xCert']
df7.head()

IDRSSD      xAsOf                             xName xState    xCert
0      37 2018-06-30            BANK OF HANCOCK COUNTY     GA  10057.0
1     242 2018-06-30  FIRST COMMUNITY BANK XENIA-FLORA     IL   3850.0
2     279 2018-06-30       MINEOLA COMMUNITY BANK, SSB     TX  28868.0
3     354 2018-06-30                  BISON STATE BANK     KS  14083.0
4     439 2007-06-30                      PEOPLES BANK     GA  16498.0

In [11]:
df7 = df7.merge(df6n[['IDRSSD','AsOf']],how='inner', left_on=['IDRSSD'], right_on=['IDRSSD'])
df7.columns = ['IDRSSD', 'xAsOf', 'xName', 'xState','xCert', 'nAsOf']
df7.tail()

IDRSSD      xAsOf                      xName xState    xCert  \
10767  5143788 2018-06-30  WINTER PARK NATIONAL BANK     FL  59105.0   
10768  5192496 2018-06-30              ENDEAVOR BANK     CA  59099.0   
10769  5193989 2018-06-30              INFINITY BANK     CA  59098.0   
10770  5205819 2018-06-30                STUDIO BANK     TN  59112.0   
10771  5227101 2018-06-30           COMMERCEONE BANK     AL  59114.0   

           nAsOf  
10767 2017-09-30  
10768 2018-03-31  
10769 2018-03-31  
10770 2018-06-30  
10771 2018-06-30

In [12]:
df5m = df5.mean()
df5m = df5m.reset_index()
df5m.drop('Cert', inplace=True, axis=1)
df8 = df7.merge(df5m, how='inner', left_on='IDRSSD', right_on='IDRSSD')

In [13]:
df8_orig = df8.copy()
len(df8) # 10762 (201612), 10766 (201709), 10767 (201712), 10772 (201806)

10772

In [14]:
df8.head()

IDRSSD      xAsOf                             xName xState    xCert  \
0      37 2018-06-30            BANK OF HANCOCK COUNTY     GA  10057.0   
1     242 2018-06-30  FIRST COMMUNITY BANK XENIA-FLORA     IL   3850.0   
2     279 2018-06-30       MINEOLA COMMUNITY BANK, SSB     TX  28868.0   
3     354 2018-06-30                  BISON STATE BANK     KS  14083.0   
4     439 2007-06-30                      PEOPLES BANK     GA  16498.0   

       nAsOf        eTotAst        aTotAst         eTotLn          eDDep  \
0 2001-03-31   81053.985714   80809.500000   39669.014286   64771.385714   
1 2001-03-31   33730.042857   34075.971429   21569.042857   25725.928571   
2 2001-03-31  152253.100000  147260.457143   74969.371429  105441.814286   
3 2001-03-31    8871.742857    8997.471429    4000.700000    7716.685714   
4 2001-03-31  166510.615385  165717.384615  118665.038462  144540.423077   

   eFDep        eTotDep  
0    0.0   64771.385714  
1    0.0   25725.928571  
2    0.0  105441.814286  
3    0.0    7716.685714  
4    0.0  144540.423077

In [15]:
df8.columns = ['IDRSSD','xAsOf','xName','xState','xCert','nAsOf','aeTotAst','aaTotAst','aeTotLn','aeDDep','aeFDep','aeTotDep']
df8.tail()

IDRSSD      xAsOf                      xName xState    xCert  \
10767  5143788 2018-06-30  WINTER PARK NATIONAL BANK     FL  59105.0   
10768  5192496 2018-06-30              ENDEAVOR BANK     CA  59099.0   
10769  5193989 2018-06-30              INFINITY BANK     CA  59098.0   
10770  5205819 2018-06-30                STUDIO BANK     TN  59112.0   
10771  5227101 2018-06-30           COMMERCEONE BANK     AL  59114.0   

           nAsOf   aeTotAst   aaTotAst  aeTotLn   aeDDep  aeFDep  aeTotDep  
10767 2017-09-30  129357.25  110667.25  57186.5  92494.0     0.0   92494.0  
10768 2018-03-31   40229.00   33595.50   6833.0  16108.0     0.0   16108.0  
10769 2018-03-31   38686.50   35105.50   2235.0   9153.5     0.0    9153.5  
10770 2018-06-30   45553.00   33282.00      0.0    370.0     0.0     370.0  
10771 2018-06-30   56804.00   51817.00   2170.0   8603.0     0.0    8603.0

In [16]:
ff = pd.read_csv("CBfails_det201806.csv")
ff2 = ff[['State', 'Year', 'Institution Name ','Cert ', 'Effective Date ']]
ff2.columns = ['State', 'Year', 'Name', 'Cert', 'EffDate']
ff2['FailDate'] = pd.to_datetime(ff2['EffDate'])
ff2 = ff2[ff2['FailDate']<'2018-06-30'] #  ONLY USE FAILDATES before Dec 2017!!!!!!!!!!!! (to align with Call Data)
ff2.dropna(subset=['Cert'],inplace=True)
ff2['Cert'] = ff2['Cert'].astype(float)
ff2.head()

State    Year                                             Name     Cert  \
0    IL  2017.0              WASHINGTON FEDERAL BANK FOR SAVINGS  30570.0   
1    KS  2017.0  THE FARMERS AND MERCHANTS STATE BANK OF ARGONIA  17719.0   
2    IL  2017.0                              FAYETTE COUNTY BANK   1802.0   
3    WI  2017.0                                    GUARANTY BANK  30003.0   
4    LA  2017.0                                   FIRST NBC BANK  58302.0   

      EffDate   FailDate  
0  12/15/2017 2017-12-15  
1  10/13/2017 2017-10-13  
2   5/26/2017 2017-05-26  
3    5/5/2017 2017-05-05  
4   4/28/2017 2017-04-28

In [17]:
dff2 = ff2[ff2.duplicated(['Cert'])][['Name','Cert']]  ## find duplicate Certs
len(dff2)  # 44

44

In [18]:
ff2lst = list(dff2['Cert'])
len(ff2lst)                      ## list of dup Certs (28, now 44) (assistance, then failure)

44

In [19]:
ff2a = ff2[ff2.Cert.isin(ff2lst)]
ff2a.sort_index(by=['Cert','FailDate'],inplace=True)  ## sorted list of dup Certs
ff2a.head()

State    Year                              Name    Cert     EffDate  \
2374    TX  1988.0        FIRST CITY, TEXAS - DALLAS  1722.0   4/20/1988   
705     TX  1992.0    FIRST CITY, TEXAS - DALLAS, NA  1722.0  10/30/1992   
2375    TX  1988.0      FIRST CITY, TEXAS - BEAUMONT  3093.0   4/20/1988   
706     TX  1992.0  FIRST CITY, TEXAS - BEAUMONT, NA  3093.0  10/30/1992   
2373    TX  1988.0         FIRST CITY, TEXAS - BRYAN  3120.0   4/20/1988   

       FailDate  
2374 1988-04-20  
705  1992-10-30  
2375 1988-04-20  
706  1992-10-30  
2373 1988-04-20

In [20]:
## sections to dedup Cert field: (saves latest date, which usually is the fail date vs assistance date)

In [21]:
ff2x = ff2[['Cert','FailDate']].groupby('Cert').max()
ff2x.reset_index(drop=False,inplace=True)
ff2x['mx'] = 'x'
ff2x.tail()

Cert   FailDate mx
3559  90257.0 1992-08-21  x
3560  90266.0 1992-09-18  x
3561  90307.0 1992-08-14  x
3562  90310.0 1991-06-07  x
3563  91189.0 1998-07-23  x

In [22]:
ff3 = ff2.merge(ff2x,how='left',left_on=['Cert','FailDate'],right_on=['Cert','FailDate'])
ff3.tail()

State    Year                             Name     Cert    EffDate  \
3603    MO  1970.0  FIRST STATE BANK OF BONNE TERRE  12227.0  8/24/1970   
3604    NJ  1970.0          EATONTOWN NATIONAL BANK  19276.0   8/7/1970   
3605    KY  1970.0       FARMERS BANK OF PETERSBURG  13164.0  6/25/1970   
3606    MI  1970.0   THE PEOPLES STATE SAVINGS BANK  14987.0  4/18/1970   
3607    IA  1970.0       STATE BANK OF PRAIRIE CITY  14571.0  2/22/1970   

       FailDate mx  
3603 1970-08-24  x  
3604 1970-08-07  x  
3605 1970-06-25  x  
3606 1970-04-18  x  
3607 1970-02-22  x

In [23]:
ff4=ff3.dropna()
len(ff3), len(ff4) # 3608, 3564

(3608, 3564)

In [24]:
df8 = df8.merge(ff4[['Cert','FailDate']], how='left', left_on='xCert', right_on='Cert')
df8 = df8.drop('Cert', axis=1)
len(df8) # 10772

10772

In [25]:
df8.sort_index(by='aeTotAst',ascending=False, inplace=True)
df8.reset_index(drop=True, inplace=True)  # renumber rows based on largest to smallest banks
df8.reset_index(drop=False, inplace=True)  # move row numbers into another column called 'index'
df8.rename(columns={'index':'Rank'},inplace=True)  # rename 'index' column to 'Rank'
df8['Rank'] = df8['Rank'] + 1  # add 1 so that largest bank has rank 1 rather than rank 0
df8.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2018-06-30  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2018-06-30      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2018-06-30           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2018-06-30     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.461295e+09  1.435970e+09  5.103033e+08  6.666457e+08   
1 2001-03-31  1.255158e+09  1.259335e+09  6.572162e+08  7.949126e+08   
2 2001-03-31  1.053310e+09  1.050354e+09  5.043212e+08  2.793764e+08   
3 2001-03-31  8.702706e+08  8.583451e+08  5.267557e+08  6.019828e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate  
0  2.535391e+08  9.201849e+08        NaT  
1  1.059607e+08  9.008733e+08 2009-01-16  
2  4.392890e+08  7.186653e+08 2008-11-23  
3  6.760383e+07  6.695866e+08        NaT  
4  2.469325e+07  2.951088e+08        NaT

In [26]:
df8['pr_eTA'] = df8['aeTotAst']*100/df8['aeTotAst'].sum()
df8['pr_eTAcs'] = df8['pr_eTA'].cumsum()
df8['pr_eTL'] = df8['aeTotLn']*100/df8['aeTotLn'].sum()
df8['pr_eTLcs'] = df8['pr_eTL'].cumsum()
df8['pr_eDD'] = df8['aeDDep']*100/df8['aeTotLn'].sum()
df8['pr_eDDcs'] = df8['pr_eDD'].cumsum()
df8.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2018-06-30  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2018-06-30      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2018-06-30           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2018-06-30     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.461295e+09  1.435970e+09  5.103033e+08  6.666457e+08   
1 2001-03-31  1.255158e+09  1.259335e+09  6.572162e+08  7.949126e+08   
2 2001-03-31  1.053310e+09  1.050354e+09  5.043212e+08  2.793764e+08   
3 2001-03-31  8.702706e+08  8.583451e+08  5.267557e+08  6.019828e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.535391e+08  9.201849e+08        NaT  9.193848   9.193848  5.638326   
1  1.059607e+08  9.008733e+08 2009-01-16  7.896924  17.090772  7.261562   
2  4.392890e+08  7.186653e+08 2008-11-23  6.626980  23.717752  5.572229   
3  6.760383e+07  6.695866e+08        NaT  5.475375  29.193127  5.820108   
4  2.469325e+07  2.951088e+08        NaT  2.778653  31.971779  2.813476   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.638326  7.365749   7.365749  
1  12.899888  8.782965  16.148713  
2  18.472117  3.086821  19.235534  
3  24.292225  6.651290  25.886824  
4  27.105701  2.987814  28.874638

In [27]:
df8.to_csv("BkAvgAstPR_FailDate_201806.csv")

In [28]:
df8_fail = df8[df8['FailDate'].map(lambda x: x.year) > 2000]   #cbt[cbt['AsOf'].map(lambda x: x.month) == 12] 
len(df8_fail) # 513

513

In [29]:
df8_fail.to_csv("FailedBkList_AvgAst_201806.csv")